In [214]:
import tensorflow as tf
import numpy as np

import tensorflow_datasets as tfds

## Data


In [215]:
mnist_dataset,mnist_info = tfds.load(name='mnist',with_info=True,as_supervised=True)

In [216]:
mnist_train, mnist_test = mnist_dataset['train'],mnist_dataset['test']

num_validation_samples = .1 * mnist_info.splits['train'].num_examples

num_validation_samples = tf.cast(num_validation_samples,tf.int64)
num_test_samples = tf.cast(mnist_info.splits['test'].num_examples,tf.int64)


In [217]:
def scale(image,label):
    image=tf.cast(image,tf.float32)
    image /= 255.
    return image,label



In [218]:
scaled_train_and_validation_data = mnist_train.map(scale)
scaled_test_data = mnist_train.map(scale)

BUFFER_SIZE = 10000


shuffled_train_and_validation = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)


validation_data = shuffled_train_and_validation.take(num_validation_samples)
train_data = shuffled_train_and_validation.skip(num_validation_samples)

In [219]:
BATCH_SIZE = 100
#We don't need to batch the validation data since we are only forward propagating, the batching gives average loss or accuracy and while validating we need the accurate result.
#But the model however expects validation data to also be batched.

train_data = train_data.batch(BATCH_SIZE)

#This simply instructs to take the whole validation samples as a single batch.
validation_data = validation_data.batch(num_validation_samples)

test_data = scaled_test_data.batch(num_test_samples)

validation_inputs,validation_targets = next(iter(validation_data))

## Model


### Outline the model


In [220]:
input_size = 784
output_size = 10
hidden_layer_size = 200

model = tf.keras.Sequential([
                                tf.keras.layers.Flatten(input_shape=(28,28,1)),
                                tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                                tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                                tf.keras.layers.Dense(output_size,activation='softmax')
])


## Choose the optimizer and the loss function

In [221]:
# custom_optimizer = tf.keras.optimizers.SGD(learning_rate=0.02)
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

## Training

In [222]:
NUM_EPOCHS = 5

model.fit(train_data,epochs=NUM_EPOCHS,validation_data=(validation_inputs,validation_targets),verbose=2)

Epoch 1/5
540/540 - 8s - loss: 0.2747 - accuracy: 0.9195 - val_loss: 0.1443 - val_accuracy: 0.9560
Epoch 2/5
540/540 - 5s - loss: 0.1085 - accuracy: 0.9674 - val_loss: 0.0875 - val_accuracy: 0.9713
Epoch 3/5
540/540 - 6s - loss: 0.0711 - accuracy: 0.9777 - val_loss: 0.0616 - val_accuracy: 0.9795
Epoch 4/5
540/540 - 5s - loss: 0.0530 - accuracy: 0.9832 - val_loss: 0.0590 - val_accuracy: 0.9827
Epoch 5/5
540/540 - 7s - loss: 0.0420 - accuracy: 0.9870 - val_loss: 0.0466 - val_accuracy: 0.9848


## Test the model

In [223]:
test_loss, test_acccuracy = model.evaluate(test_data)

6/6 [==============================] - 5s 863ms/step - loss: 0.0321 - accuracy: 0.9901


In [225]:
print('Test loss: {0:.2f}. Test accuracy: {1: .2f}%'.format(test_loss,test_acccuracy*100.))

Test loss: 0.03. Test accuracy:  99.01%
